In [1]:

import numpy as np
import pandas as pd

matrix = np.array([
    [0.191, 0.081, 0.067, 0.111],
    [0.082, 0.059, 0.063, 0.102],
    [0.036, 0.083, 0.076, 0.088],
    [0.029, 0.088, 0.235, 0.078],
    [0.306, 0.055, 0.113, 0.068],
    [0.038, 0.154, 0.033, 0.144],
    [0.014, 0.100, 0.047, 0.148],
    [0.029, 0.155, 0.139, 0.105],
    [0.014, 0.083, 0.150, 0.089], 
    [0.010, 0.142, 0.067, 0.069]
])

criteria = ['C1', 'C2', 'C3', 'C4']
sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.16376103, 0.29605439, 0.46393757, 0.076247])

# Normalize the decision matrix
def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

# ELECTRE I method
def electre(matrix, criteria_types, weights, sites, criteria, c_thresh=0.6, d_thresh=0.4):
    n, m = matrix.shape
    norm_matrix = normalize_matrix(matrix, criteria_types)
    weighted_matrix = norm_matrix * weights

    # Concordance matrix
    concordance = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            mask = weighted_matrix[i] >= weighted_matrix[j]
            concordance[i, j] = weights[mask].sum()

    # Discordance matrix
    discordance = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            diff = np.abs(weighted_matrix[i] - weighted_matrix[j])
            denom = diff.max() if diff.max() != 0 else 1
            disc = np.max([
                (weighted_matrix[j, k] - weighted_matrix[i, k]) / denom
                if weighted_matrix[i, k] < weighted_matrix[j, k] else 0
                for k in range(m)
            ])
            discordance[i, j] = disc

    # Outranking relation
    c_bar = c_thresh
    d_bar = d_thresh
    outranking = (concordance >= c_bar) & (discordance <= d_bar)

    # Kernel (non-dominated alternatives)
    kernel = []
    for i in range(n):
        if not outranking[:, i].any():
            kernel.append(sites[i])

    # Concordance dominance count
    concordance_dominance = outranking.sum(axis=1)

    # Net dominance = Dominance - Dominated
    net_dominance = outranking.sum(axis=1) - outranking.sum(axis=0)

    # Prepare DataFrame
    df = pd.DataFrame({
        'Site': sites,
        'Concordance Dominance': concordance_dominance,
        'Net Dominance': net_dominance,
        'Kernel Member': [site in kernel for site in sites]
    })
    df['Rank (Concordance)'] = df['Concordance Dominance'].rank(method='min', ascending=False).astype(int)
    df['Rank (Net Dominance)'] = df['Net Dominance'].rank(method='min', ascending=False).astype(int)
    df = df.set_index('Site')
    df = df.sort_values('Rank (Net Dominance)')

    return df, concordance, discordance, outranking

# Run ELECTRE I
result_df, concordance, discordance, outranking = electre(matrix, criteria_types, weights, sites, criteria)

# Print results
print("ELECTRE I Results:")
print(result_df)

print("\nConcordance Matrix:\n", pd.DataFrame(concordance, index=sites, columns=sites))
print("\nDiscordance Matrix:\n", pd.DataFrame(discordance, index=sites, columns=sites))
print("\nOutranking Relation Matrix:\n", pd.DataFrame(outranking, index=sites, columns=sites))


ELECTRE I Results:
        Concordance Dominance  Net Dominance  Kernel Member  \
Site                                                          
Site4                       5              5           True   
Site8                       5              5           True   
Site9                       3              2          False   
Site5                       1              1           True   
Site1                       1              0          False   
Site10                      1              0          False   
Site6                       0             -1          False   
Site3                       1             -2          False   
Site7                       0             -3          False   
Site2                       0             -7          False   

        Rank (Concordance)  Rank (Net Dominance)  
Site                                              
Site4                    1                     1  
Site8                    1                     1  
Site9               